In [20]:
import pandas as pd
import numpy as np
import json
import os
import torch

import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from torch.utils.data import random_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# DATASET

In [21]:
train_image_folder = './normal/train/all'
train_class_file = 'train_name2class_PH.json'

test_image_folder = './normal/test/all'
test_class_file = 'test_name2class_PH.json'

batch = 32
device = 'cuda'

In [22]:
class ImageDataset(Dataset):
    def __init__(self, image_folder, class_file, transform=None):
        self.image_folder = image_folder
        self.class_file = class_file
        self.transform = transform


        self.image_files = []
        self.classes = []
        
        import json

        with open(class_file, 'r') as f:
            name2class = json.load(f)

        for f in os.listdir(self.image_folder):
            self.image_files.append(os.path.join(self.image_folder, f))
            
            
            if '_' in f:
                self.classes.append(name2class[str(f.split('_')[0])])
            else:
            

                self.classes.append(name2class[str(f.split('.')[0])])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_file = self.image_files[idx]
        image = Image.open(image_file)
        label = self.classes[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [23]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0) == 1 else x)
])

train_dataset = ImageDataset(train_image_folder, train_class_file, transform=transform)

val_size = int(0.1 * len(train_dataset))  
train_size = len(train_dataset) - val_size

train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=True)

test_dataset = ImageDataset(test_image_folder, test_class_file, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=True)


In [24]:
len(train_loader), len(val_loader), len(test_loader)

(43, 5, 16)

# TRAIN MODEL

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []

class CNN_PH(nn.Module):
    def __init__(self):
        super(CNN_PH, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(10 * 30 * 30, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.relu(self.conv3(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.sigmoid(x)
        return x



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_PH().to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device) 
        optimizer.zero_grad()
        outputs = model(inputs)  
        labels = labels.float().view(-1, 1)
        loss = criterion(outputs, labels) 
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # Проверка
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)  
            labels = labels.float().view(-1, 1)  
            loss = criterion(outputs, labels) 
            val_loss += loss.item()
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    train_loss_history.append(running_loss)
    val_loss_history.append(val_loss)
    val_acc_history.append(correct/total)
    print(f"Epoch {epoch+1}, Loss: {running_loss}, Val_loss: {val_loss}, accuracy: {correct/total}")

In [7]:
model_save_path = 'aug_100.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Модель сохранена в {model_save_path}")

Модель сохранена в aug_100.pth


In [19]:
import torch
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

model = CNN_PH()  

model.load_state_dict(torch.load('aug_100.pth',  ))
model.to(device)

model.eval()

true_labels = []
pred_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:  
        inputs, labels = inputs.to(device), labels.to(device)  
        outputs = model(inputs) 
        predicted = (outputs > 0.5).float()  

        true_labels.extend(labels.cpu().numpy())  
        pred_labels.extend(predicted.cpu().numpy()) 

true_labels = np.array(true_labels)
pred_labels = np.array(pred_labels)

f1 = f1_score(true_labels, pred_labels)
recall = recall_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels)
accuracy = accuracy_score(true_labels, pred_labels)

print(f"F1 Score: {f1}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"Accuracy: {accuracy}")


F1 Score: 0.7670103092783506
Recall: 0.744
Precision: 0.7914893617021277
Accuracy: 0.774
